## Grover's Algorithm Homework

### Quantum Computing FRI Stream Fall 2019

In [1]:
#Import the necessary tools for the computation
from math import pi
import numpy as np
from qiskit import*


In [2]:
def CZ(qc,a,Cr,Tr):
    if len(Cr)<=1:
        if len(Cr)==1: 
        #apply the C-gate
            qc.cu1(np.pi*a,Cr[0],Tr)
        else:                   
        #just apply the gate
            qc.u1(np.pi*a,Tr)
    else:
        nm=len(Cr)
        #apply C-sqrt(U)
        CZ(qc,a/2,[Cr[nm-1]],Tr)
       
        #apply CCNot
        qc.h(Cr[nm-1])
        CZ(qc,1,Cr[0:nm-1],Cr[nm-1])
        qc.h(Cr[nm-1])
        
        #apply C-sqrt(U dg)
        CZ(qc,-a/2,[Cr[nm-1]],Tr)
        
        #apply CCNot
        qc.h(Cr[nm-1])
        CZ(qc,1,Cr[0:nm-1],Cr[nm-1])
        qc.h(Cr[nm-1])
        
        #apply CC-sqrt(U)
        CZ(qc,a/2,Cr[0:nm-1],Tr)

        
def Tof(qc,Cr,Tr):
    qc.h(Tr)
    CZ(qc,1,Cr,Tr)
    qc.h(Tr)

### 1. Use your recursive controlled Z gate to construct your oracle

In [3]:
# Assign the key values of your function
# n is the number of input qubits
# a is the solution to the equation f(a)=1, the number you are trying to find.
# a should be an n bit number

In [4]:
n = int(input("number of input qubits: "))
N = n**2
a = 2

number of input qubits: 4


In [5]:
# Build your unitary U_f as a Python function
# As a reminder, it should give output=1 for input=a and output=0 for all other inputs

In [6]:
def Ufgate(n, a):
    for y in range(1,n+1):
        if y != a:
            qc.x(q[y])
    
    Tof(qc, Toffoli, q[0])
        
    for y in range(1,n+1):
        if y != a:
            qc.x(q[y])
    

### 2. Make the necessary preparations to implement Grover's search

In [7]:
# Build your W gate
# this rotates your state toward the solution

In [8]:
def Wgate(n):
    for y in range(1,n+1):
        qc.h(q[y])
        qc.x(q[y])
        
    CZ(qc, 1, Cr, q[1])
        
    for y in range(1, n+1):
        qc.x(q[y])
        qc.h(q[y])

In [9]:
# calculate the number R which is the number of times to apply the algorithm

In [10]:
R = int((pi*(N)**0.5)//4)



### 3. Initialize your circuit and apply the algorithm

In [11]:
#Initialize your registers
#Quantum registers: we want 1 output and n inputs
#Classical registers: we want at least n to hold the outcome of the final measurement
q=QuantumRegister(n+1)
c=ClassicalRegister(n)
qc=QuantumCircuit(q,c)
Cr = []
Toffoli = []
for y in range(2,n+1):
    Cr.append(q[y])
for y in range(1, n+1):
    Toffoli.append(q[y])

In [12]:
# initialize the output register to |1>
# then Hadamard the output to put it in |->=(|0>-|1>)/sqrt(2)

In [13]:
qc.x(q[0])
qc.h(q[0])

In [14]:
# Apply a Hadamard gate to the input registers

In [15]:
#iterate through and apply a hadamard gate in each position
for y in range(1,n+1):
    qc.h(q[y])

In [16]:
# In a loop which executes R times:
#  apply your unitary U_f
#  and then your W

In [17]:
for y in range(R):
    Ufgate(n, a)
    Wgate(n)

In [18]:
# Apply a Hadamard and then a NOT gate to the output register

In [19]:
qc.h(q[0])
qc.x(q[0])

In [20]:
#apply measurement gates to input registers

In [21]:
for y in range(1, n+1):
    qc.measure(y, 1)


### 4. Run the simulation, and interpret the outputs

In [22]:
#For fun, draw the circuit!
qc.draw()

In [23]:
#Run the simulation - ecexute this cell
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result()

In [24]:
#Print the outcome of the measurement
counts=result.get_counts(qc)
print(counts)

{'0000': 1007, '0010': 17}
